In [1]:
import datacube
import pandas as pd
from odc.ui import DcViewer
# Set some configurations for displaying tables nicely
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_rows', None)

In [15]:
!pip install openpyxl

     |████████████████████████████████| 249 kB 2.7 MB/s eta 0:00:01


In [1]:
import pandas as pd
# df = pd.read_excel("africa-2019.xlsx", sheet_name = "Sheet1")
df = pd.read_excel("x-y-for presence0.xlsx")
df_coordinates = df[['x', 'y']]
df_coordinates.head(5)

,x,y
0,37.875,3.125
1,38.875,2.125
2,36.625,5.875
3,34.625,1.125
4,39.125,7.375


In [2]:
%matplotlib inline

import datacube
import xarray as xr
import matplotlib.pyplot as plt

from deafrica_tools.plotting import display_map
from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.datahandling import load_ard

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


In [3]:
dc = datacube.Datacube(app='cropland_extent')

In [4]:
lat, lon = 1.349722, 32.944444

buffer = 0.0005

resolution=(-10, 10)

#join lat, lon, buffer to get bounding box
lat_range = (lon - buffer, lon + buffer)
lon_range = (lat + buffer, lat - buffer)
print(lat_range)
df_coordinates['lat_range'] = list(zip(df.y - buffer, df.y + buffer))
df_coordinates['lon_range'] = list(zip(df.x - buffer, df.x + buffer))


(32.943943999999995, 32.944944)


In [5]:
time_period = ('2019')

list_lat_range = list(df_coordinates['lat_range'])
list_lon_range = list(df_coordinates['lon_range'])
crop_extent = []

for lat_range, lon_range in zip(list_lat_range, list_lon_range):
    query = {
        'time': time_period,
        'x': lat_range,
        'y': lon_range,
        'resolution':resolution
    }
    
    # now load the crop-mask using the query
    cm = dc.load(product='crop_mask',
             **query).squeeze()
    if cm.isnull() == False:
        crop_extent.append(cm.prob.mean(skipna=True).values)
    else:
        crop_extent.append("None")

df_coordinates['crop_extent'] = crop_extent
df_coordinates.head(5)

,x,y,lat_range,lon_range,crop_extent
0,37.875,3.125,"(3.1245, 3.1255)","(37.8745, 37.8755)",None
1,38.875,2.125,"(2.1245, 2.1255)","(38.8745, 38.8755)",None
2,36.625,5.875,"(5.8745, 5.8755)","(36.6245, 36.6255)",255.0
3,34.625,1.125,"(1.1245, 1.1255)","(34.6245, 34.6255)",42.892561983471076
4,39.125,7.375,"(7.3745, 7.3755)","(39.1245, 39.1255)",None


In [6]:
df_coordinates.to_csv("crop_extent-presence-0.csv")

In [75]:
ds = load_ard(dc=dc, 
              products=['s2_l2a'],
              time=('2019-01', '2019-12'), 
              measurements=['red', 'nir'],
              mask_filters=(['opening',5],['closing', 5]), #improve cloud mask
              min_gooddata=0.35,
              like=cm.geobox
             )
    

Using pixel quality parameters for Sentinel 2
Finding datasets
    s2_l2a
Counting good quality pixels for each time step
Filtering to 106 out of 142 time steps with at least 35.0% good quality pixels
Applying morphological filters to pq mask (['opening', 5], ['closing', 5])
Applying pixel quality/cloud mask
Loading 106 time steps


/usr/local/lib/python3.8/dist-packages/rasterio/warp.py:346: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(


In [76]:
ds = calculate_indices(ds, 'NDVI', drop=True, satellite_mission='s2')

Dropping bands ['red', 'nir']


In [78]:
ds = ds.where(cm.prob > 50)
ds

<xarray.Dataset>
Dimensions:      (time: 106, y: 14, x: 11)
Coordinates:
  * time         (time) datetime64[ns] 2019-01-12T09:04:20 ... 2019-12-28T09:...
  * y            (y) float64 -1.833e+06 -1.833e+06 ... -1.833e+06 -1.833e+06
  * x            (x) float64 1.864e+06 1.864e+06 ... 1.865e+06 1.865e+06
    spatial_ref  int32 6933
Data variables:
    NDVI         (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan
Attributes:
    crs:           EPSG:6933
    grid_mapping:  spatial_ref

In [7]:
dc = datacube.Datacube(app="Landcover_Classification")

In [8]:
lat, lon = 1.349722, 32.944444

buffer = 0.004

resolution=(-10, 10) #resample so we can view a large area

measurements='classification'

#convert the lat,lon,buffer into a range
lons = (lon - buffer, lon + buffer)
lats = (lat - buffer, lat + buffer)

In [ ]:
time_period = ('2019')

list_lat_range = list(df_coordinates['lat_range'])
list_lon_range = list(df_coordinates['lon_range'])
class_ = []
cci_class = []

for lat_range, lon_range in zip(list_lat_range, list_lon_range):
    query = {
        'time': ('2019'),
        'x': lat_range,
        'y': lon_range,
        'resolution':resolution,
        'output_crs': 'epsg:6933',
        'measurements':measurements
    }
    # now load the crop-mask using the query
    ds_io = dc.load(product='io_lulc', **query).squeeze()
    if ds_io.isnull().all() == False:
        class_.append((ds_io.to_array().values[0][0][0]))
    else:
        class_.append("None")       
    

In [ ]:
time_period = ('2019')

list_lat_range = list(df_coordinates['lat_range'])
list_lon_range = list(df_coordinates['lon_range'])
class_ = []
cci_class = []

for lat_range, lon_range in zip(list_lat_range, list_lon_range):
    query = {
        'time': ('2019'),
        'x': lat_range,
        'y': lon_range,
        'resolution':resolution,
        'output_crs': 'epsg:6933',
        'measurements':measurements
    }
    # now load the crop-mask using the query
    ds_io = dc.load(product='io_lulc', **query).squeeze()
    ds_cgls = dc.load(product='cgls_landcover', time='2019', measurements=measurements, like=ds_io.geobox).squeeze()
    if ds_cgls.isnull().all() == False:
        cci_class.append((ds_cgls))
    else:
        cci_class.append("None")
        

In [13]:
mode_array = []
import scipy
for array in class_:
    mode = scipy.stats.mode(array)
    mode_array.append(format(mode[0][0]))
# for array in cci_class:
    
df_coordinates['mode_class'] = mode_array
df_coordinates.to_csv('df-mode-crop-presence-0.csv')

/tmp/ipykernel_927/2028145915.py:4: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode = scipy.stats.mode(array)
/usr/local/lib/python3.8/dist-packages/scipy/stats/_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
/tmp/ipykernel_927/2028145915.py:4: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  mode = 

In [108]:
ds_cgls = dc.load(product='cgls_landcover', time='2019', measurements=measurements, like=ds_io.geobox).squeeze()

print(ds_cgls)

<xarray.Dataset>
Dimensions:         (y: 103, x: 78)
Coordinates:
    time            datetime64[ns] 2019-07-02T11:59:59.500000
  * y               (y) float64 1.727e+05 1.727e+05 ... 1.717e+05 1.717e+05
  * x               (x) float64 3.178e+06 3.178e+06 ... 3.179e+06 3.179e+06
    spatial_ref     int32 6933
Data variables:
    classification  (y, x) uint8 40 40 40 40 40 40 40 ... 40 40 40 40 40 40 126
Attributes:
    crs:           EPSG:6933
    grid_mapping:  spatial_ref
